**VENDA DE APARTAMENTOS EM NATAL-RN (OLX)**

Descrição dos atributos:
1. URL;
2. Título do anúncio;
3. Endereço;
4. Valor da taxa de condomínio;
4. Valor da taxa de condomínio;
5. Área do apartamento;
6. Quantidade de quartos;
7. Quantidade de suítes;
8. Quantidade de banheiros;
9. Quantidade de vagas no estacionamento;
10. Preço do apartamento (valor de venda).

bonus:
12. data da publicação do anúncio.


In [ ]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from datetime import datetime, timedelta
import re
import csv
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#open connection and grab page code
#webpage = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/venda/apartamentos' #page 2, add to the end: ?o=2
#uClient = urlopen(webpage)
#page_html = soup(uClient)
#items = page_html.findAll("li", {"data-parent_category_id":"1000"})
#items[1]

In [ ]:
year = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Abr': '04', "Mai": '05', 'Jun': '06', 'Jul': '07', 'Ago': '08', 'Set': '09', 'Out': '10', 'Nov': '11', 'Dez': '12'}

URLS = [] ; names = [] ; restRooms = [] ; rooms = []; sizes = []; taxes = []; parkings = []; regions = []; prices = []; days = []

for i in range(20): #n pages, 50 items/page
  if i == 0:
    webpage = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/venda/apartamentos'
  else: #page 2 to i
     webpage = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/venda/apartamentos'+'?o='+str(i+1)
  
  uClient = urlopen(webpage)
  page_html = soup(uClient)
  items = page_html.findAll("li", {"data-parent_category_id":"1000"})

  print("Page:", i+1)
   
  for item in items:
    descricao = item.findAll("div", {"class":"OLXad-list-line-1 mb5px"})
    #attribute 1
    URLS.append(webpage)
    title = descricao[0].h2.text.strip()
    #attribute 2
    names.append(title) #strip to remove \n
    #attribute 3
    try: 
      restRoom = re.search('\d (?=suítes)|\d (?=suites)|\d (?=suíte)|\d (?=suite)', title)
      restRooms.append(restRoom.group(0).strip())
    except:
      restRooms.append(None)

    detail = descricao[0].p.text.strip()
    treat_detail = detail.replace("\t","").replace(" | ", " ")  
    
    #attribute 4
    room = re.search('\d (?=quartos)|\d (?=quarto)', treat_detail)
    rooms.append(room.group(0).strip())
  
    #attribute 5
    try: 
      size = re.search('\d{1,5} (?=m²)', treat_detail)
      sizes.append(size.group(0).strip())
    except:
      sizes.append(None)
    #attribute 6
    try: 
      tax = re.search('R\$\s\d{1,5}', treat_detail)
      taxes.append(tax.group(0).replace('R$ ', ''))
    except: 
      taxes.append(None)
    
    #attribute 7
    try:
      park = re.search('\d (?=vagas)|\d (?=vaga)', treat_detail)
      parkings.append(park.group(0).strip())
    except:
      parkings.append(None)
    
    descricao2 = items[1].findAll("div", {"class":"OLXad-list-line-2"})
    region = descricao2[0].p.text.strip()
    treat_region = region.split(',')
    #attribute 8
    regions.append(treat_region[1].strip())
    #attribute 9
    descricao3 = items[1].findAll("div", {"class":"col-3"})
    prices.append(descricao3[0].p.text.strip().replace("R$", "").replace(" ",""))
    #attribute 10
    descricao4 = items[1].findAll("div", {"class":"col-4"})
    day = descricao4[0].p.text.strip()

    if day == "Hoje":
      # dd/mm/YY
      days.append(datetime.today().strftime('%d/%m/%y%y'))
    elif day == "Ontem":
      day = datetime.now() - timedelta(days=1)
      days.append(day.strftime('%d/%m/%y%y')  )
    else:
      dateWritten = day.split(' ')
      month = year[dateWritten[1]]
      days.append(dateWritten[0]+'/'+month+'/2020')

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Page: 9
Page: 10
Page: 11
Page: 12
Page: 13
Page: 14
Page: 15
Page: 16
Page: 17
Page: 18
Page: 19
Page: 20


In [ ]:
with open('drive/My Drive/Residencia/venda-apartamento-natal.csv', 'w') as csvFile:
  writer = csv.writer(csvFile)
  writer.writerow(["URL, Título, Suítes, Quartos, Tamanho, Condomínio, Estacionamento, Bairro, Preço, Data"])
  for URL, name, restRoom, room, size, taxe, parking, region, price, day in zip(URLS, names, restRooms, rooms, sizes, taxes, parkings, regions, prices, days):
    writer.writerow([URL + ', ' + name + ', ' + str(restRoom) + ', ' + room + ', ' + str(size) + ', ' + str(taxe) + ', ' + str(parking) + ', ' + region + ', ' + price + ', ' + day])